<a href="https://colab.research.google.com/github/andreaafs/MVP-Machine-Learning-Analytics/blob/main/MVP_Risck_Heart_Disease_final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP: Previsão de Risco de Doenças Cardíacas

Autor: Andrea Ferreira dos Santos

Data: 25/09/2025

Matrícula: 40530010056_20250_01

## 1. Escopo, Objetivo e Definição do Problema
- Objetivo: prever risco de doenças cardíacas (classificação binária: sim/não)
- Hipóteses iniciais: idade, pressão, colesterol e histórico familiar influenciam mais.
- Restrições: dataset público, sem dados clínicos sensíveis.

O conjunto de dados foi obtido por amostragem e adaptação do conjunto de dados original do Kaggle. Cada linha no conjunto de dados corresponde a um indivíduo diferente. Algumas colunas são as respostas a perguntas feitas aos entrevistados sobre seu estado de saúde, como:

"Você tem sérias dificuldades para andar ou subir escadas?" ou "Você fumou pelo menos 100 cigarros em toda a sua vida? O alvo é a variável Doença Cardíaca, que indica se os entrevistados que já relataram ter doença coronariana (DAC) ou infarto do miocárdio.


***OBS:*** Foram excluídas muitas linhas, pois não consegui carregar o dataset completo para o guithub


## 2. Reprodutibilidade e Ambiente

In [ ]:
!pip install dython

In [ ]:
!pip install cartopy


In [ ]:
!apt-get install -y wkhtmltopdf
!pip install pdfkit


In [ ]:
# Importação de pacotes
import pandas as pd  # Series and DataFrames
import numpy as np  # linear algebra
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import math
import random
import torch
import time
import cartopy

import shap
from scipy.stats import ks_2samp
import nbformat
from nbconvert import HTMLExporter
import pdfkit
import os

# nltk
import nltk
import ast
import random
import warnings
import cartopy
warnings.filterwarnings("ignore", category=DeprecationWarning)

from matplotlib import cm
from plotly.subplots import make_subplots
from sklearn import model_selection # model assesment and model selection strategies
from sklearn import metrics # model evaluation metrics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    roc_auc_score, log_loss,confusion_matrix, RocCurveDisplay
)
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from warnings import simplefilter
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from warnings import simplefilter




simplefilter(action='ignore', category=UserWarning)

# visualization
from wordcloud import WordCloud
colors = ['#494BD3', '#E28AE2', '#F1F481', '#79DB80', '#DF5F5F',
              '#69DADE', '#C2E37D', '#E26580', '#D39F49', '#B96FE3']
from plotly.offline import init_notebook_mode, iplot

from dython.nominal import associations

pd.set_option('display.max_columns', None)
# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format


# Print last updated
import time
print(f"Last updated: {time.asctime()}")

## 3. Dados: Carga, Entendimento e Qualidade

In [ ]:
def read_dataset():
    try:
        # Carregar dataset direto do GitHub
        url = "https://raw.githubusercontent.com/andreaafs/MVP-Machine-Learning-Analytics/refs/heads/main/heart_disease_data2.csv"

        df = pd.read_csv(url)

        print('"heart_disease_data2.csv" imported successfully!')
        # Exibir as primeiras linhas do dataframe

        return df

    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado: {e}")
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")

In [ ]:
df =read_dataset()

In [ ]:
# Conferir colunas e renomear target caso necessário
print(df.columns)

# Definir a primeira coluna 'HeartDisease' como target
target_col = df.columns[0]
df.rename(columns={target_col: 'target'}, inplace=True)

print(df.info())
print(df.describe())
print(df.isnull().sum())
sns.countplot(x='target', data=df)
plt.show()

In [ ]:
df.head(5).T

In [ ]:
df.tail(5).T

In [ ]:
df.describe().T

**ANALISANDO OS DADOS**

In [ ]:
df.info(verbose=False)

In [ ]:
df.columns

**Descrição da tabela:**

Explicação Variável



*   HeartDisease (Target): Entrevistados que já relataram ter tido doença cardíaca coronária (DCC) ou infarto do miocárdio (IM)
*   BMI: Índice de Massa Corporal (IMC)
*   Smoking: Você já fumou pelo menos 100 cigarros em toda a sua vida? [Nota: 5 maços = 100 cigarros]
*   AlcoholDrinking:Bebedores pesados ​​(homens adultos que consomem mais de 14 doses por semana e mulheres adultas que consomem mais de 7 doses por semana)
*  Stroke: (Já lhe disseram) que teve um AVC?
*  PhysicalHealth:Agora pensando na sua saúde física
*  MentalHealth: Pensando na sua saúde mental
Dificuldade para Caminhar: Você tem muita dificuldade para caminhar ou subir escadas?
*  Sex: Você é homem ou mulher?
*  AgeCategory:Categoria etária de quatorze níveis
*  Race:Valor de raça/etnia imputado
*  Diabetic: (Já lhe disseram) que teve diabetes?
*  PhysicalActivity: Adultos que relataram ter praticado atividade física ou exercício nos últimos 30 dias, além do seu trabalho regular
*  GenHealth: Você diria que, em geral, sua saúde é Boa/Muito Boa/…
*  SleepTime: Em média
*  Asthma:(Já lhe disseram) que teve asma?
*  KidneyDisease: Não Incluindo cálculos renais
*  SkinCancer: (Já lhe disseram que você tinha) câncer de pele?
*  HeartDisease_FamilyHistory: Você tem histórico familiar de doença cardíaca?
*  State: US sate \EUA (residência)


In [ ]:
df.head(4)

In [ ]:
print("Total de linhas do dataframe : ",df.shape[0])


**Relatório rápido sobre dados**

In [ ]:
print(df.describe(include=['object']).T)

#### Podemos obsersevar nos dados apresentados acima que  temos:
  * Mais mulheres;
  * Menos fumantes;
  * Faixa etária entre 65-69;
  * Raça Branca;
  * Estado predominante OH;
  * Condições de Saúde é muito boa
    
    OBS: As respostas obtidas não represemtam uma amostragem com uma variação de casos, onde podemos ter uma analise não representariva. Com a sequencias dos anos, esperamos que as respostas sejam mais representativas.

In [ ]:
print(df.isnull().sum())

Durante a etapa de análise de qualidade dos dados, foram identificados dois campos com proporções muito elevadas de valores ausentes:

AlcoholDrinking com 101.504 valores nulos

HeartDisease_FamilyHistory com 270.420 valores nulos

Esses atributos foram considerados não informativos neste momento e poderam ser removidos do dataset, a fim de evitar viés e perda de amostra.

In [ ]:
# ---- Distribuição do target ----
sns.countplot(x='target', data=df)
plt.title("Distribuição do Target")
plt.show()

In [ ]:
# ---- Histogramas das variáveis numéricas ----
df.hist(bins=20, figsize=(12,10))
plt.suptitle("Distribuição das variáveis numéricas")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Seleciona apenas as colunas numéricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Configura a figura
fig, axes = plt.subplots(nrows=len(num_cols)//3 + 1, ncols=3, figsize=(15, 5 * (len(num_cols)//3 + 1)))
axes = axes.flatten()

# Loop para plotar cada coluna
for i, col in enumerate(num_cols):
    axes[i].hist(df[col], bins=20, color='skyblue', edgecolor='black')
    axes[i].set_title(f'Distribuição de {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequência')

# Remove subplots extras vazios
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle("Histogramas das Variáveis Numéricas", fontsize=16, y=1.02)
plt.show()


**Gráfico de Densidade**

In [ ]:
df.plot(kind = 'density', subplots = True, layout = (3,3), sharex = False, figsize = (15,10))
plt.show()

In [ ]:
# Boxplot
df.plot(kind = 'box', subplots = True, layout = (3,3), sharex = False, sharey = False, figsize = (15,10))
plt.show()

**Podemos obsvar que temos bastentes outliers visíveis como pontos fora do alcance dos bigodes. **

In [ ]:
# ---- Associações mistas (numéricas e categóricas) ----
associations(df, figsize=(12, 10))
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.select_dtypes(include=['int64','float64']).corr(),
            annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de Correlação - Variáveis Numéricas")
plt.show()


**scatter plot)**

In [ ]:
sns.pairplot(df)


 Descritivas estatísticas do conjunto de dados

#### variaveis categoricas

In [ ]:
df.describe(include=['object']).T

In [ ]:
for index in df.select_dtypes(include = "object"):
    print(f"{index}")
    print(df[index].unique())
    print('\n')

#### variaveis não categoricas

In [ ]:
df.select_dtypes(include=['number']).describe().T

In [ ]:
for index in df.select_dtypes(include = "number"):
    print(f"{index}")
    print(df[index].unique())
    print('\n')

Número de valores **nulos**

In [ ]:
df.select_dtypes(include=['number']).isnull().sum()

In [ ]:
df.select_dtypes(include=['object']).isnull().sum()



**Distribuição/Intervalo de Valores Alvo**

In [ ]:
df[['target']].head()

In [ ]:
# Acessar a coluna 'HeartDisease'
coluna_heart_disease = df['target']
print("Coluna target:")
print(coluna_heart_disease)

In [ ]:
print('This is a Series')
print(type(df['target']))
df['target'].head()

In [ ]:
df['target'].value_counts(dropna=False)

A variavel Target possui dois valors e um nulo

HeartDisease = "Yes"

HeartDisease = "No"

In [ ]:
# Vamos ver quantas amostras positivas (1) e negativas (0) temos em nosso dataframe
df.target.value_counts()

In [ ]:
# Plot as contagens de valores com um gráfico de barras
df.target.value_counts().plot(kind="bar", color=["salmon", "lightblue"]);

**Obs:**
Como foi encontrado um valor NaN a opção mais comum é remover as linhas com NaN na target, porque sem o valor correto não tem como treinar o modelo (nem validar a previsão).

In [ ]:
df = df.dropna(subset=["target"])

In [ ]:
df['target'].value_counts(dropna=False)

In [ ]:
df.info()

Como noasa variavel Target é categorica, para faciitar o estudo irei transformar em não categoria variando os valores entre 0 e 1

HeartDisease = "Yes" => tem chance de ter problemas cardiacos será 1

HeartDisease = "No" => Não tem chance de ter problemas cardiacos será 0

In [ ]:
df["target"] = df["target"].map({"No": 0, "Yes": 1}).astype("Int64")

In [ ]:
# Acessar a coluna 'HeartDisease'
coluna_target = df['target']
print("Coluna target:")
print(coluna_target)

In [ ]:
# retorna o total de linhas
df['target'].count()

In [ ]:
#  retorma a soma
df['target'].sum()

In [ ]:
# retorna a media
df['target'].mean()

Distribuição de variáveis ​​e relações com o Target

* Variavel BMI

para ver a relação e distribuição da variavel com o Targer



In [ ]:
print("BMI")
print(df['BMI'].unique())
print('\n')

In [ ]:
df.BMI.value_counts()

In [ ]:
# Compare target column with BMI column
pd.crosstab(df.target, df.BMI)

In [ ]:
# plot
pd.crosstab(df.target, df.BMI).plot(kind="bar",
                                    figsize=(10,6),
                                    color=["salmon", "lightblue"]);

**Histograma da BMI**

In [ ]:
# Plotar o histograma usando Matplotlib
plt.figure(figsize=(10, 6))
sns.histplot(df['BMI'], bins=10, kde=True, label='BMI Distribution')
plt.xlabel('BMI')
plt.ylabel('Frequency')
plt.title('Histogram of BMI')
plt.legend(title='Legend')  # Adiciona uma legenda com título
plt.show()

#### Este gráfico evidencia que a maior parte da amostra se concentra em valores de IMC entre 20 e 40, indicando que a maioria dos indivíduos analisados encontra-se em condição de sobrepeso ou obesidade. Além disso, observa-se que justamente nessas faixas o risco de desenvolver doenças cardíacas tende a ser mais elevado, reforçando a relação entre excesso de peso e maior probabilidade de problemas cardiovasculares.

**Distribuição dos valores dos valores de BMI entre os indivíduos no seu conjunto de dados**

In [ ]:
df_bmi = df[['BMI','target']].groupby(['BMI']).mean()

In [ ]:
df_bmi.head()

In [ ]:
df_bmi

In [ ]:
plt.figure(figsize=(14, 7))
sns.lineplot(data=df_bmi, x='BMI', y='target')
plt.xlabel('BMI')
plt.ylabel('Mean Heart Disease')
plt.title('Mean Heart Disease by BMI')
plt.show()

#### Neste gráfico, observa-se que a média de pessoas com maior incidência de doenças cardíacas está concentrada justamente na faixa que, anteriormente, já havia se destacado pela maior frequência da amostra: indivíduos em situação de sobrepeso ou obesidade. Além disso, nota-se a presença de alguns valores destoantes, provavelmente resultantes de erros de registro ou inconsistências na base de dados, já que representam medidas que seriam inviáveis de serem alcançadas na prática

**Analise**

In [ ]:
plt.figure(figsize=(14, 7))
sns.scatterplot(data=df_bmi, x='BMI', y='target')
plt.xlabel('BMI')
plt.ylabel('Mean Heart Disease')
plt.title('Mean Heart Disease by BMI')
plt.show()

**Calculando a medias por faixa etaria  de ter problemas cardiaco**

In [ ]:
df_agecategory = df.groupby('AgeCategory')['target'].mean().reset_index()

In [ ]:
print(df_agecategory)

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
from google.colab import data_table
data_table.disable_dataframe_formatter()

In [ ]:
data_table.DataTable(df_agecategory, include_index=False, num_rows_per_page=10)

In [ ]:
plt.figure(figsize=(14, 7))

# Criar uma paleta com cores diferentes para cada barra
palette = sns.color_palette("tab10", n_colors=len(df_agecategory))

sns.barplot(
    data=df_agecategory,
    x='AgeCategory',
    y='target',
    palette=palette
)

plt.xlabel('Age Category')
plt.ylabel('Mean Heart Disease')
plt.title('Mean Heart Disease by Age Category')
plt.xticks(rotation=90)
plt.show()

##### Pela media podemos afirmar que com o aumento da idade temos mais probabilidade de ter doenças cardiacas. Visualmente podemos observar que  depois deis 50 o Risco tem um aumento siginificativo.

**Calculando a media por faixa etaria, sexo e ter ou nao Problemas cardiacos**

In [ ]:
df_sex_agecategory = df[['AgeCategory', 'Sex', 'target']].groupby(['AgeCategory', 'Sex']).mean().reset_index()

In [ ]:
print(df_sex_agecategory)

In [ ]:
data_table.DataTable(df_sex_agecategory, include_index=False, num_rows_per_page=30)

In [ ]:
plt.figure(figsize=(14, 7))
sns.barplot(data=df_sex_agecategory, x='AgeCategory', y='target', hue='Sex')
plt.xlabel('Age Category')
plt.ylabel('Mean Heart Disease')
plt.title('Mean Heart Disease by Age Category and Sex')
plt.xticks(rotation=90)
plt.show()

Podemos observar que os homens possuem uma maior probabilidade de ter doenças cardiacas independendente da idade e que a partir da faixa de 45 -49 o risco aumenta muito

In [ ]:
df_sex_agecategory.info(verbose=False)

In [ ]:
df_agecategory.info(verbose=False)

In [ ]:
plt.figure(figsize=[9,3])
df_agecategory['target'].plot()

Verificamos conforme o aumento da idade temos mais risco de ter Doença cardíaca

#### Variavel  Race

Usaremos o histograma para distribuição da variação em relação a raça e como é uma variavel categorica irei transformar em não categorica

In [ ]:
df['Race'].head()

In [ ]:
valores_distintos = df['Race'].unique()
print(valores_distintos)

In [ ]:
df["Race"].isnull().sum()

In [ ]:
df.loc[:, "Race"] = (
    df["Race"]
    .replace({
        "White": 0,
        "Black": 1,
        "Asian": 2,
        "American Indian/Alaskan Native": 3,
        "Other": 4,
        "Hispanic": 5,
        "nan": -1
    })
    .fillna(-1)  # cobre NaN reais
    .astype("int8")  # converte para numérico compacto
)



In [ ]:
df['Race'].head()

In [ ]:
df_race = df[['Race','target']].groupby(['Race']).count()
print(df_race)

In [ ]:
df['Race'].plot(kind='hist', bins = 100)

##### Verificamos que a raça não é uma opção boa para analise pois notamos que a nossa amostragem e composta de sua maioria da raça branca. Logo, poderemos ver uma certa tendencica a informar que a raça branca terá maior probabilidade de ter doenças cardiácas

**Calculando a media por Raça**

In [ ]:
df['Race'].value_counts()

In [ ]:
df['Race'].value_counts(normalize=True) * 100

In [ ]:
df.groupby('Race')['target'].mean()


In [ ]:
# Calcular média de HeartDisease por raça
race_mean = df.groupby('Race')['target'].mean().reset_index()

# Criar paleta de cores (uma para cada raça)
palette = sns.color_palette("Set2", n_colors=len(race_mean))

# Plotar gráfico de barras
plt.figure(figsize=(12,6))
sns.barplot(data=race_mean, x='Race', y='target', palette=palette)
plt.xlabel('Race')
plt.ylabel('Mean Heart Disease')
plt.title('Mean Heart Disease by Race')
plt.xticks(rotation=45)
plt.show()

Pelo gráfico histograma de Raça.

*  Podemos notar que a maior frequÊncia da raça é a Branca
*  Asiáticos demonstram menor probabilidade de ter doenças cardiacas enquanto os nativos americanos apresentam a maior



**Relação da Raca/ BMI e HeartDisease**

In [ ]:
# Calculando médias por raça
mean_values_by_race = df.groupby('Race').agg({
    'BMI': 'mean',
    'target': 'mean'
}).reset_index()

print(mean_values_by_race)

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=mean_values_by_race, x='BMI', y='target', hue='Race', palette='Set2', s=100)
plt.xlabel('Mean BMI')
plt.ylabel('Mean Heart Disease')
plt.title('Mean BMI vs Mean Heart Disease by Race')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

# Plotagem de BMI
plt.subplot(1, 2, 1)
sns.barplot(data=mean_values_by_race, x='Race', y='BMI', palette='Blues')
plt.xlabel('Race')
plt.ylabel('Mean BMI')
plt.title('Mean BMI by Race')

# Plotagem de HeartDisease
plt.subplot(1, 2, 2)
sns.barplot(data=mean_values_by_race, x='Race', y='target', palette='Oranges')
plt.xlabel('Race')
plt.ylabel('Mean Heart Disease')
plt.title('Mean Heart Disease by Race')

plt.tight_layout()
plt.show()

Ao comparar o IMC (BMI) entre as diferentes raças, observa-se que os valores apresentam diferenças entre os grupos. Por outro lado, as probabilidades de desenvolver doenças cardíacas apresentam variações mais significativas, sugerindo que o IMC tem uma correlação maior com o risco de doenças cardíacas neste conjunto de dados do que a Raça.

Variavel MentalHealth

Analise para MentalHealth

In [ ]:
df['MentalHealth'].head()

In [ ]:
df['MentalHealth'].plot(kind='hist', bins = 100)

In [ ]:
# Calcular a média de MentalHealth por HeartDisease
mean_mental_health_by_heart_disease = df.groupby('target')['MentalHealth'].mean().reset_index()
print(mean_mental_health_by_heart_disease)

In [ ]:
plt.figure(figsize=(17, 6))
sns.barplot(data=mean_mental_health_by_heart_disease, x='target', y='MentalHealth', palette='Set2')
plt.xlabel('Heart Disease')
plt.ylabel('Mean Mental Health')
plt.title('Mean Mental Health by Heart Disease')
plt.xticks([0, 1], ['No Heart Disease', 'Heart Disease'])
plt.ylim(0, 60)  # Ajustar limite do eixo y se necessário
plt.show()

##### Notamos que as pessoas que possuem uma saude metal possuem uma pequena diferença em não ter problemas cardiacos, essa relação sozinha mostra que não é o unico fator a ser considerado

**Analise da variavel Diabete, para isso terei que transformar a variavel em não categorica**

In [ ]:
# Checar se existem NaNs
print("Quantidade de NaNs:", df["Diabetic"].isna().sum())

In [ ]:
# Verificar se restaram valores não convertidos
print("Valores únicos em Diabetic:", df["Diabetic"].unique())

In [ ]:
df_Diabetic = df.groupby('Diabetic').size().reset_index(name='count')
print(df_Diabetic)

In [ ]:
df['Diabetic'] = df['Diabetic'].replace({
    'No': 0,
    'No, borderline diabetes': 0,
    'Yes': 1,
    'Yes (during pregnancy)': 1,
    'nan': -1
})


In [ ]:
df['Diabetic'].head()

In [ ]:
df['Diabetic'].plot(kind='hist', bins = 100)

In [ ]:
df_diabetic = df[['Diabetic', 'target']].groupby(['Diabetic']).mean().reset_index()

In [ ]:
data_table.DataTable(df_diabetic, include_index=False, num_rows_per_page=10)

In [ ]:
plt.figure(figsize=(17, 6))
sns.barplot(data=df_diabetic, x='target', y='Diabetic', hue='target', palette='Set2')
plt.xlabel('Heart Disease')
plt.ylabel('Mean Diabetic')
plt.title('Mean Diabetic by Heart Disease')
plt.xticks([0, 1], ['No Heart Disease', 'Heart Disease'])
plt.ylim(0, 60)
plt.legend([],[], frameon=False)  # Remove legenda se não quiser mostrar
plt.show()



Pelo grafico podemos informar que as pessoas que tem Diabete tem mais probalbilidade de ter problemas cardiacos

##### Escolhemos agora a variabe BMI - IMC

In [ ]:
df['BMI'].plot(kind='hist', bins = 100)

Podemos notar que esta variavel possui uma boa distribuição

In [ ]:
## Without target
sns.boxplot(data=df, y='BMI')

In [ ]:
# Plotar o boxplot
plt.figure(figsize=(17,6))
sns.boxplot(data=df, x='target', y='BMI')
plt.xlabel('Heart Disease')
plt.ylabel('BMI')
plt.title('BMI Distribution by Heart Disease Status')
plt.xticks([0, 1], ['No Heart Disease', 'Heart Disease'])
plt.grid(True)  # Adicionar grades ao gráfico
plt.tight_layout()  # Ajustar layout para melhor visualização
plt.show()

#### O gráfico mostra uma distribuição proporcional entre pessoas com e sem doenças cardíacas, com uma leve tendência de indivíduos com a doença apresentarem IMC ligeiramente mais elevado

In [ ]:
df[['BMI','target']].groupby('target').describe()

Noto que os valores apresentados que o ICM sozinho não determinar a um risco maior de ter doenças cardiacas

**Variaveis PhysicalHealth & MentalHealth**

In [ ]:
# Selecionar apenas as colunas de interesse
cols = ['PhysicalHealth', 'MentalHealth', 'target']

# Calcular correlação com o target
correlations = df[cols].corr()['target'].drop('target')

# Plotar gráfico de barras
plt.figure(figsize=(17,6))
sns.barplot(x=correlations.index, y=correlations.values, palette='Set2')
plt.ylabel('Correlação com Doença Cardíaca')
plt.title('Contribuição de PhysicalHealth e MentalHealth para Doença Cardíaca')
plt.ylim(0,1)
plt.show()

In [ ]:
df[df.target == 0]['PhysicalHealth'].plot(kind='hist', alpha=.5, color="green")
df[df.target == 1]['MentalHealth'].plot(kind='hist', alpha=.5, color="orange")

In [ ]:
df[df.target == 0]['PhysicalHealth'].plot(kind='hist', alpha=.5, color="green", density=True)
df[df.target == 1]['MentalHealth'].plot(kind='hist', alpha=.5, color="orange", density=True)

#### A barra mais alta indica qual variável tem maior correlação com o risco de doenças cardíacas.

Podemos observar que PhysicalHealth, significa que a saúde física tem mais impacto no risco.


###Variaveis Categoricas

**Variável Estado (State)**

In [ ]:
plt.figure(figsize=[17,3])
plt.xticks(rotation=90)
sns.countplot(data=df, x='State', hue='target')

aqui apresenta-se a distribuição de pessoas com ou sem doenças cardiacas por estado

In [ ]:
df.pivot_table(index='State', values='target', aggfunc=[len, sum, np.mean]).sort_values(by=[('mean','target')], ascending = False)

In [ ]:
pd.crosstab(df['State'], df['target'])

**Variavel Asthma**

In [ ]:
plt.figure(figsize=[17,6])
plt.xticks(rotation=90)
sns.countplot(data=df, x='Asthma', hue='target')

Preliminarmente podemos avaliar que o grafico apresenta dados que nos informam que as pessoas que não tem asma são a maioria com problemas cardiacos

In [ ]:
df_asma_mean = df[['Asthma','target']].groupby(['Asthma']).mean()
print(df_asma_mean)

In [ ]:
df.select_dtypes(include=['object']).isnull().sum()

In [ ]:
# Dados originais em um formato pivot
data = {
    'Asthma': ['No', 'Yes'],
    'HeartDisease': [0.08, 0.12]
}

df_asma_mean = pd.DataFrame(data)

# Verificar as colunas do DataFrame
print("Colunas do DataFrame:", df_asma_mean.columns)

# Verifique se as colunas 'HeartDisease' (target) e 'Asthma' existem
if 'HeartDisease' in df_asma_mean.columns and 'Asthma' in df_asma_mean.columns:
    plt.figure(figsize=[17,6])
    plt.xticks(rotation=90)
    sns.barplot(data=df_asma_mean, x='Asthma', y='HeartDisease')
    plt.xlabel('Asthma')
    plt.ylabel('Mean Heart Disease')
    plt.title('Mean Heart Disease by Asthma Status')
    plt.show()
else:
    print("As colunas 'HeartDisease' e/ou 'Asthma' não existem no DataFrame")

O eixo x mostra o status de asma (Asthma) com duas categorias: No (não tem asma) e Yes (tem asma).

O eixo y mostra a média de ocorrência de doenças cardíacas (HeartDisease) para cada grupo.

Cada barra representa a probabilidade média de doenças cardíacas em pessoas com ou sem asma.

Pessoas sem asma (No) têm uma média de doenças cardíacas de 0,08 (8%).

Pessoas com asma (Yes) têm uma média de doenças cardíacas de 0,12 (12%).

Isso indica que, neste conjunto de dados, ter asma está associado a um risco ligeiramente maior de desenvolver doenças cardíacas.

#### Variavel AlcoholDrinking

In [ ]:
plt.figure(figsize=[17,6])
plt.xticks(rotation=90)
sns.countplot(data=df, x='target', hue='AlcoholDrinking')

Analisando o grafico e vendo a correlação de pessoas com doenças cardiacas e o consumo de alcool, denota-se um nmero quase irrisorio de pessoas que tomam alcool e tem doenças cardiacas. Tal amostra por ser muito discrepante pode mostrar certo problema na propria amostra, talvez com pessoas tendo vegonha ou medo de admitir consumir alcool ja tendo uma doença cardiaca

In [ ]:
df_drinhing_mean = df[['AlcoholDrinking','target']].groupby(['AlcoholDrinking']).mean()
print(df_drinhing_mean)


In [ ]:
# Dados originais em um formato pivot
data_drink =df_drinhing_mean


df_drinhing_mean = pd.DataFrame(data_drink)

# Verificar as colunas do DataFrame
print("Colunas do DataFrame:", df_drinhing_mean.columns)

# Verifique se as colunas 'HeartDisease' e 'AlcoholDrinking' existem
if 'HeartDisease' in df_drinhing_mean.columns and 'AlcoholDrinking' in df_drinhing_mean.columns:
    plt.figure(figsize=[17,6])
    plt.xticks(rotation=90)
    sns.barplot(data=df_drinhing_mean, x='AlcoholDrinking', y='HeartDisease')
    plt.xlabel('Alcohol Drinking')
    plt.ylabel('Mean Heart Disease')
    plt.title('Mean Heart Disease by Alcohol Drinking Status')
    plt.show()
else:
    print("As colunas 'HeartDisease' e/ou 'AlcoholDrinking' não existem no DataFrame")

###### E neste grafico, um surpreendente diferença que mostra que pessoas que não consumem alcool tendem a ter quase o dobro a mais de chance de ter doenças cardiacas (de 5 para 9 por cento), com a diferença percentual absoluta tambem sendo de 4 por cento.  Porem, essa analise não pode ser condiderada como verdadeira, devido ao alto numero de falantes, ou seja, sem informação para a coluna.

#### **4. Definição do target, variáveis e divisão dos dados**

####**Preparação de Dados**

In [ ]:
# devido ter alterados os valores das variaveis estou carregndo novamente
df =read_dataset()

In [ ]:
df.columns

Nulos nas variaveis categoricas

In [ ]:
df.select_dtypes(include=['object']).isnull().sum()

Como não é possível determinar com segurança como preencher os valores nulos — o uso da média ou da moda poderia gerar vieses e impactar negativamente a análise, visto que algumas features podem ser sensíveis e alterar o comportamento do modelo, a solução adotada será criar uma categoria adicional para representar os valores nulos. Embora fosse possível excluir as linhas com nulos, não há garantia de como isso afetaria as previsões. Entretanto, a coluna HeartDisease_FamilyHistory, por apresentar muitos valores ausentes, será removida da análise, enquanto os registros com valor nulo na coluna target serão excluídos.

In [ ]:
# Exclusão de coluna com muitos valores faltantes
df.drop(columns=['HeartDisease_FamilyHistory'], inplace=True)


In [ ]:
df.columns

In [ ]:
# Renomear a coluna HeartDisease para target
target_col = 'HeartDisease'  # ou df.columns[0] se for a primeira coluna
df.rename(columns={target_col: 'target'}, inplace=True)

In [ ]:
df.select_dtypes(include=['object']).isnull().sum()

In [ ]:
df.select_dtypes(include=['object']).isnull().sum()

In [ ]:
df.head(2)

In [ ]:
print("Antes:", len(df))
df.dropna(subset=["target"], inplace=True)
print("Depois:", len(df))


In [ ]:
# Transformação de tipos
df['target'] = df['target'].replace({'No':0,'Yes':1}).astype(int)

In [ ]:
# ==================================
# 1. Definição de features e target
# ==================================

target_col = "target"

features = [col for col in df.columns if col != target_col]

X = df[features]
y = df[target_col]


In [ ]:
# ==================================
# 2. Identificar colunas numéricas e categóricas
# ==================================
num_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

print("Numéricas:", num_features)
print("Categóricas:", cat_features)

In [ ]:
# ==================================
# 3. Pré-processamento
# ==================================
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)

In [ ]:
print(preprocessor)

# ==================================
# 4. Construir pipeline reprodutível
# ==================================

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_validate

# ===============================
# Seed para reprodutibilidade
# ===============================
SEED = 42

# Modelos candidatos
modelos_candidates = {
    "Dummy": DummyClassifier(strategy="most_frequent"),
    # Serve como baseline: sempre prediz a classe mais frequente
    "SVC": SVC(probability=True, random_state=SEED),
     # Classificador linear ou não-linear, bom para problemas de alta dimensão
    "LogReg": LogisticRegression(max_iter=500, random_state=SEED),
     # Modelo linear interpretável, rápido e robusto
    "KNN": KNeighborsClassifier(n_neighbors=5),
     # Baseado em vizinhos, útil quando classes estão próximas em clusters
    "DecisionTree": DecisionTreeClassifier(max_depth=5, random_state=SEED),
     # Fácil de interpretar, captura relações não-lineares
    "RandomForest_light": RandomForestClassifier(
        n_estimators=50, max_depth=5, random_state=SEED, n_jobs=-1
    )
     # Ensemble de árvores, reduz overfitting e melhora robustez

}

# Validação cruzada
# - Avalia o desempenho do modelo em diferentes divisões do treino, evitando overfitting
# - Permite comparar modelos de forma justa antes de treinar no conjunto completo
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

scoring = {
    "accuracy": "accuracy",
    "f1": "f1_weighted",
    "roc_auc": "roc_auc",
    "precision": "precision_weighted",
    "recall": "recall_weighted"
}

# Coluna alvo 'target'
target = "target"

# Separar features e target
X = df.drop(columns=[target])
y = df[target]


# Divisão treino/teste (opcional, mas recomendado)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)


# Lista para armazenar resultados
resultados_rapidos = []

for nome, modelo in modelos_candidates.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("modelo", modelo)
    ])

    scores = cross_validate(
        pipe,
        X_train,
        y_train,
        cv=cv,
        scoring=scoring,
        return_train_score=False
    )

    # Média das métricas de validação cruzada
    media_scores = {m: round(v.mean(), 3) for m, v in scores.items() if "test_" in m}
    media_scores["modelo"] = nome
    resultados_rapidos.append(media_scores)



# Transformar em DataFrame e mostrar
df_resultados = pd.DataFrame(resultados_rapidos)
print(df_resultados)


In [ ]:
# ===============================
# 8. Seleção do melhor modelo (ex: pelo F1)
# ===============================
melhor_nome = df_resultados.sort_values("test_f1", ascending=False).iloc[0]["modelo"]
melhor_modelo = modelos_candidates[melhor_nome]
print(f"Melhor modelo selecionado: {melhor_nome}")

# ===============================
# 9. Treinar o melhor modelo no treino completo
# ===============================
pipe_final = Pipeline([
    ("preprocessor", preprocessor),
    ("modelo", melhor_modelo)
])

pipe_final.fit(X_train, y_train)

# ===============================
# 10. Avaliação no teste (opcional)
# ===============================
y_pred = pipe_final.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


In [ ]:
# ===============================
# Seed
# ===============================
SEED = 42

# ===============================
# Separar features e target
# ===============================
target = "target"  # substituir pelo nome da coluna alvo
X = df.drop(columns=[target])
y = df[target]

# ===============================
# Divisão treino/teste
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# ===============================
# Preprocessor
# ===============================
colunas_numericas = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
colunas_categoricas = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), colunas_numericas),
        ("cat", OneHotEncoder(handle_unknown="ignore"), colunas_categoricas)
    ]
)

# ===============================
#  Modelos candidatos e grids
# ===============================
modelos_candidates = {
    "Dummy": {
        "modelo": DummyClassifier(strategy="most_frequent"),
        "param_grid": {}
    },
    "LogReg": {
        "modelo": LogisticRegression(max_iter=500, random_state=SEED),
        "param_grid": {
            "modelo__C": [0.01, 0.1, 1, 10]
        }
    },
    "SVC": {
        "modelo": SVC(probability=True, random_state=SEED),
        "param_grid": {
            "modelo__C": [0.1, 1, 10],
            "modelo__kernel": ["linear", "rbf"]
        }
    },
    "KNN": {
        "modelo": KNeighborsClassifier(),
        "param_grid": {
            "modelo__n_neighbors": [3,5,7]
        }
    },
    "DecisionTree": {
        "modelo": DecisionTreeClassifier(random_state=SEED),
        "param_grid": {
            "modelo__max_depth": [3,5,10,None]
        }
    },
    "RandomForest_light": {
        "modelo": RandomForestClassifier(random_state=SEED, n_jobs=-1),
        "param_grid": {
            "modelo__n_estimators": [50,100],
            "modelo__max_depth": [5,10,None]
        }
    }
}

# ===============================
# Validação cruzada e GridSearchCV
# Motivo da validação cruzada:
# - Avalia o desempenho do modelo em diferentes divisões do treino, evitando overfitting
# - Permite comparar modelos de forma justa antes de treinar no conjunto completo
# ===============================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
scoring = {
    "accuracy": "accuracy",
    "f1": "f1_weighted",
    "roc_auc": "roc_auc",
    "precision": "precision_weighted",
    "recall": "recall_weighted"
}

resultados_grid = []

for nome, info in modelos_candidates.items():
    print(f"\nExecutando GridSearchCV para {nome}...")
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("modelo", info["modelo"])
    ])

    grid = GridSearchCV(
        estimator=pipe,
        param_grid=info["param_grid"],
        cv=cv,
        scoring=scoring,
        refit="f1",  # escolher o melhor pelo F1 ponderado
        n_jobs=-1,
        verbose=0
    )

    grid.fit(X_train, y_train)

    resultados_grid.append({
        "modelo": nome,
        "melhores_params": grid.best_params_,
        "melhor_f1": round(grid.best_score_, 3),
        "melhor_estimador": grid.best_estimator_
    })

# ===============================
# 8. Mostrar resultados
# ===============================
df_grid_results = pd.DataFrame(resultados_grid)
print("\nResultados GridSearchCV para todos os modelos:\n")
print(df_grid_results[["modelo", "melhores_params", "melhor_f1"]])


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

print("\n================ Avaliação no conjunto de teste ================\n")

for resultado in resultados_grid:
    nome = resultado["modelo"]
    pipe = resultado["melhor_estimador"]

    # Previsões no teste
    y_pred = pipe.predict(X_test)

    # Probabilidades (necessário para ROC-AUC)
    if hasattr(pipe.named_steps["modelo"], "predict_proba"):
        y_proba = pipe.predict_proba(X_test)[:, 1]  # Para a classe positiva
        roc_auc = round(roc_auc_score(y_test, y_proba), 3)
    else:
        roc_auc = None

    print(f"Modelo: {nome}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    if roc_auc:
        print(f"ROC-AUC: {roc_auc}")
    print("-" * 60)


In [ ]:
# ===============================
# Seed e CV
# ===============================
SEED = 42
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
scoring = {
    "accuracy": "accuracy",
    "f1": "f1_weighted",
    "roc_auc": "roc_auc",
    "precision": "precision_weighted",
    "recall": "recall_weighted"
}

# ===============================
# Preprocessor
# ===============================
colunas_numericas = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
colunas_categoricas = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), colunas_numericas),
        ("cat", OneHotEncoder(handle_unknown="ignore"), colunas_categoricas)
    ]
)

# ===============================
# Modelos candidatos e grids
# ===============================
modelos_candidates = {
    "Dummy": {
        "modelo": DummyClassifier(strategy="most_frequent"),
        "param_grid": {}
    },
    "LogReg": {
        "modelo": LogisticRegression(max_iter=500, random_state=SEED),
        "param_grid": {"modelo__C": [0.01, 0.1, 1, 10]}
    },
    "SVC": {
        "modelo": SVC(probability=True, random_state=SEED),
        "param_grid": {"modelo__C": [0.1,1,10], "modelo__kernel": ["linear","rbf"]}
    },
    "KNN": {
        "modelo": KNeighborsClassifier(),
        "param_grid": {"modelo__n_neighbors": [3,5,7]}
    },
    "DecisionTree": {
        "modelo": DecisionTreeClassifier(random_state=SEED),
        "param_grid": {"modelo__max_depth": [3,5,10,None]}
    },
    "RandomForest_light": {
        "modelo": RandomForestClassifier(random_state=SEED, n_jobs=-1),
        "param_grid": {"modelo__n_estimators": [50,100], "modelo__max_depth": [5,10,None]}
    }
}

# ===============================
# GridSearchCV para todos os modelos
# ===============================
resultados_grid = []

for nome, info in modelos_candidates.items():
    print(f"\nExecutando GridSearchCV para {nome}...")

    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("modelo", info["modelo"])
    ])

    grid = GridSearchCV(
        estimator=pipe,
        param_grid=info["param_grid"],
        cv=cv,
        scoring=scoring,
        refit="f1",
        n_jobs=-1,
        verbose=0
    )

    grid.fit(X_train, y_train)

    resultados_grid.append({
        "modelo": nome,
        "melhores_params": grid.best_params_,
        "melhor_f1": round(grid.best_score_,3),
        "melhor_estimador": grid.best_estimator_
    })

df_grid_results = pd.DataFrame(resultados_grid)
print("\nResultados GridSearchCV resumidos:\n")
print(df_grid_results[["modelo","melhores_params","melhor_f1"]])

# ===============================
# Avaliação no conjunto de teste
# ===============================
f1_test = []

print("\n================ Avaliação no conjunto de teste ================\n")
for resultado in resultados_grid:
    nome = resultado["modelo"]
    pipe = resultado["melhor_estimador"]

    y_pred = pipe.predict(X_test)

    # ROC-AUC se disponível
    if hasattr(pipe.named_steps["modelo"], "predict_proba"):
        y_proba = pipe.predict_proba(X_test)[:,1]
        roc_auc = round(roc_auc_score(y_test, y_proba),3)
    else:
        roc_auc = None

    f1_val = f1_score(y_test, y_pred, average="weighted")
    f1_test.append({"modelo": nome, "f1_test": f1_val})

    print(f"Modelo: {nome}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    if roc_auc:
        print(f"ROC-AUC: {roc_auc}")
    print("-"*60)

# ===============================
# Gráfico comparativo do F1 no teste
# ===============================
df_f1_test = pd.DataFrame(f1_test)

plt.figure(figsize=(10,6))
sns.barplot(data=df_f1_test, x="modelo", y="f1_test", palette="viridis")
plt.title("F1 ponderado no conjunto de teste por modelo")
plt.ylabel("F1 ponderado")
plt.xlabel("Modelo")
plt.ylim(0,1)
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Preparar dados
df_plot = ranking_final.copy()
df_plot["ROC_AUC_plot"] = df_plot["ROC_AUC"].fillna(0)  # preencher NaN para plotar

plt.figure(figsize=(12,6))
sns.set(style="whitegrid")

# Barras de F1
sns.barplot(
    x="Modelo", y="F1", hue="Fase", data=df_plot,
    palette="pastel", edgecolor="black"
)

# Adicionar labels de F1 sobre as barras
for i, row in df_plot.iterrows():
    plt.text(i, row.F1 + 0.01, f"{row.F1:.2f}", ha='center', fontsize=9)

# Linhas de ROC AUC
for i, row in df_plot.iterrows():
    if row.ROC_AUC_plot > 0:
        plt.plot([i-0.2, i+0.2], [row.ROC_AUC_plot, row.ROC_AUC_plot], color='red', lw=2)
        plt.text(i, row.ROC_AUC_plot + 0.01, f"{row.ROC_AUC_plot:.2f}", ha='center', fontsize=9, color='red')

plt.title("Ranking de Modelos: F1 (barras) e ROC AUC (linha vermelha)")
plt.ylabel("F1 / ROC AUC")
plt.xlabel("Modelo")
plt.ylim(0,1.05)
plt.show()


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=ranking_final)

In [ ]:
import joblib
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, RocCurveDisplay

# ========================
# Seleção do melhor modelo
# ========================
melhor_modelo_nome = ranking_final.iloc[0]["Modelo"]  # topo do ranking
melhor_modelo = modelos_candidates[melhor_modelo_nome]

print(f"🏆 Melhor modelo selecionado: {melhor_modelo_nome}")

# ========================
# Treinar modelo no conjunto completo
# ========================
melhor_modelo.fit(X_train, y_train)
y_pred = melhor_modelo.predict(X_test)

# ========================
# Métricas
# ========================
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
print(f"Accuracy: {acc:.3f}")
print(f"F1 Score: {f1:.3f}")

# ROC AUC apenas se binário
if len(np.unique(y_test)) == 2 and hasattr(melhor_modelo, "predict_proba"):
    y_proba = melhor_modelo.predict_proba(X_test)[:,1]
    roc_auc = roc_auc_score(y_test, y_proba)
    print(f"ROC AUC: {roc_auc:.3f}")
else:
    roc_auc = None

# ========================
# Matriz de Confusão
# ========================
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
plt.title(f"Matriz de Confusão - {melhor_modelo_nome}")
plt.xlabel("Predito")
plt.ylabel("Real")
plt.show()

# ========================
# Curva ROC
# ========================
if roc_auc is not None:
    RocCurveDisplay.from_estimator(melhor_modelo, X_test, y_test)
    plt.show()

# ========================
# Salvar modelo
# ========================
joblib.dump(melhor_modelo, f"melhor_modelo_{melhor_modelo_nome}.joblib")
print(f"✅ Modelo salvo como melhor_modelo_{melhor_modelo_nome}.joblib")


1️⃣ Accuracy: 0.916

Significa que 91,6% das previsões do modelo estão corretas no conjunto de teste.

Accuracy mede a proporção de acertos gerais, mas não leva em conta desequilíbrio entre classes

2️⃣ F1 Score: 0.890

F1 combina precisão (precision) e revocação (recall) em uma única métrica:

𝐹
1
=
2
⋅
𝑝
𝑟
𝑒
𝑐
𝑖
𝑠
𝑖
𝑜
𝑛
⋅
𝑟
𝑒
𝑐
𝑎
𝑙
𝑙
𝑝
𝑟
𝑒
𝑐
𝑖
𝑠
𝑖
𝑜
𝑛
+
𝑟
𝑒
𝑐
𝑎
𝑙
𝑙
F1=2⋅
precision+recall
precision⋅recall
	​


Valor 0.890 indica bom equilíbrio entre evitar falsos positivos e falsos negativos.

Geralmente, em problemas com classes desbalanceadas, o F1 é mais confiável que Accuracy para medir performance real do modelo.

3️⃣ ROC AUC: 0.833

ROC AUC mede a capacidade do modelo de separar corretamente as classes:

0.5 → aleatório

1.0 → perfeita separação

0.833 é uma separação muito boa, indicando que, se variar o threshold de decisão, o modelo consegue distinguir bem entre positivos e negativos.

Modelos lineares (como Logistic Regression)

Coeficientes (coef_) indicam importância e direção do efeito da variável:

Positivo → aumenta o risco

Negativo → diminui o risco

Para visualizar de forma clara:

In [ ]:
# X_train é o usado no treino
features_treino = X_train.columns

coefs = pd.Series(melhor_modelo.coef_[0], index=features_treino)
coefs_sorted = coefs.sort_values(key=abs, ascending=False)
print(coefs_sorted.head(10))



In [ ]:


features_treino = X_train.columns
coefs = melhor_modelo.coef_[0]

# Criar DataFrame com coeficientes
df_coefs = pd.DataFrame({
    "Feature": features_treino,
    "Coeficiente": coefs
})

# Adicionar coluna de interpretação
df_coefs["Interpretacao"] = df_coefs["Coeficiente"].apply(lambda x: "Aumenta risco" if x > 0 else "Reduz risco")

# Adicionar importância absoluta para ordenar
df_coefs["AbsCoef"] = df_coefs["Coeficiente"].abs()

# Ordenar pelas features mais importantes
df_top = df_coefs.sort_values(by="AbsCoef", ascending=False).head(10)
print(df_top)

# Plotar gráfico de barras com cores
plt.figure(figsize=(10,6))
colors = ['red' if x > 0 else 'green' for x in df_top["Coeficiente"]]

sns.barplot(x="Coeficiente", y="Feature", data=df_top, palette=colors)

# Adicionar interpretação ao lado das barras
for index, row in df_top.iterrows():
    plt.text(row["Coeficiente"] + 0.02 if row["Coeficiente"] > 0 else row["Coeficiente"] - 0.02,
             index,
             row["Interpretacao"],
             color='black',
             va='center')

plt.title("Top 10 Features do Modelo com Interpretação")
plt.xlabel("Coeficiente (positivo = aumenta risco, negativo = reduz risco)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


In [ ]:
# Selecionar apenas as colunas que interessam para a tabela final
df_tabela = df_top[["Feature", "Coeficiente", "Interpretacao"]].reset_index(drop=True)

# Exibir a tabela
print(df_tabela)


| Feature                  | Coeficiente | Interpretação                                     |
| ------------------------ | ----------- | ------------------------------------------------- |
| AgeCategory\_80 or older | 1.19        | Idade ≥80 aumenta fortemente o risco.             |
| GenHealth\_Excellent     | -0.93       | Boa saúde percebida diminui o risco.              |
| AgeCategory\_75-79       | 0.84        | Idade 75-79 aumenta risco, menos que ≥80.         |
| Stroke\_Yes              | 0.77        | Histórico de AVC aumenta risco.                   |
| Stroke\_No               | -0.75       | Sem histórico de AVC diminui risco.               |
| AgeCategory\_70-74       | 0.70        | Idade 70-74 aumenta risco.                        |
| GenHealth\_Poor          | 0.66        | Saúde percebida ruim aumenta risco.               |
| GenHealth\_Fair          | 0.62        | Saúde razoável aumenta risco, mas menos que ruim. |
| AgeCategory\_35-39       | -0.55       | Adultos jovens têm risco menor.                   |
| AgeCategory\_18-24       | -0.52       | Jovens adultos têm risco ainda menor.             |

In [ ]:
# Se usar  ColumnTransformer ou pipeline
# se 0 melhor_modelo foi treinado com X_train_transformado
coefs = pd.Series(melhor_modelo.coef_[0], index=X_train.columns)
coefs_sorted = coefs.abs().sort_values(ascending=False)

print("✅ Variáveis mais importantes para o risco:")
print(coefs_sorted.head(10))



In [ ]:
# ========================
# Garantir features corretas usadas no treino
# ========================
features_treino = X_train.columns

# ========================
# Coeficientes do Logistic Regression
# ========================
coefs = pd.Series(melhor_modelo.coef_[0], index=features_treino)
coefs_sorted = coefs.sort_values(key=abs, ascending=False)

# Mostrar top 10 features
top_features = coefs_sorted.head(10)
print("🏆 Top 10 fatores que influenciam o risco de doenças cardíacas:")
print(top_features)

# ========================
# Gráfico
# ========================
plt.figure(figsize=(8,6))
colors = ['red' if x > 0 else 'green' for x in top_features]  # vermelho aumenta risco, verde diminui
top_features.sort_values().plot(kind='barh', color=colors)
plt.xlabel("Coeficiente")
plt.title("Top 10 fatores que influenciam o risco de doenças cardíacas")
plt.grid(axis='x')
plt.show()


#### cross-validated metrics

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, roc_auc_score

model = RandomForestClassifier(random_state=42)

scoring = {
    'accuracy': 'accuracy',
    'f1': make_scorer(f1_score),            # binária
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'roc_auc': make_scorer(roc_auc_score)
}

results = cross_validate(model, X, y, cv=5, scoring=scoring)
print("Accuracy por fold:", results['test_accuracy'])
print("F1 por fold:", results['test_f1'])
print("Precision por fold:", results['test_precision'])
print("Recall por fold:", results['test_recall'])
print("ROC AUC por fold:", results['test_roc_auc'])


## 8. Avaliação Final, Análise de Erros e Limitações

In [ ]:
best_model = melhor_modelo  # já treinado anteriormente

y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1]))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.show()
RocCurveDisplay.from_estimator(best_model, X_test, y_test)
plt.show()

In [ ]:
features_treino = X_train.columns

# Para modelos lineares
if hasattr(best_model, "coef_"):
    coefs = pd.Series(best_model.coef_[0], index=features_treino)
    coefs_sorted = coefs.sort_values(key=abs, ascending=False)
    top_features = coefs_sorted.head(10)
    print("🏆 Top 10 fatores que influenciam o risco:")
    print(top_features)

    plt.figure(figsize=(8,6))
    colors = ['red' if x > 0 else 'green' for x in top_features]
    top_features.sort_values().plot(kind='barh', color=colors)
    plt.xlabel("Coeficiente")
    plt.title("Top 10 fatores que influenciam o risco de doenças cardíacas")
    plt.grid(axis='x')
    plt.show()

# Para modelos de árvore
elif hasattr(best_model, "feature_importances_"):
    importances = pd.Series(best_model.feature_importances_, index=features_treino)
    importances_sorted = importances.sort_values(ascending=False)
    top_features = importances_sorted.head(10)

    plt.figure(figsize=(8,6))
    top_features.sort_values().plot(kind='barh', color='skyblue')
    plt.xlabel("Importância")
    plt.title("Top 10 features importantes (RandomForest/GBM)")
    plt.gca().invert_yaxis()
    plt.show()


Gráfico comparativo de todos os modelos (F1 e ROC AUC)

In [ ]:
df_plot = ranking_final.copy()
df_plot["ROC_AUC_plot"] = df_plot["ROC_AUC"].fillna(0)

plt.figure(figsize=(12,6))
sns.barplot(x="Modelo", y="F1", data=df_plot, palette="pastel", edgecolor="black")

# Labels F1
for i, row in df_plot.iterrows():
    plt.text(i, row.F1 + 0.01, f"{row.F1:.2f}", ha='center', fontsize=9)

# Linha ROC AUC
for i, row in df_plot.iterrows():
    if row.ROC_AUC_plot > 0:
        plt.plot([i-0.2, i+0.2], [row.ROC_AUC_plot]*2, color='red', lw=2)
        plt.text(i, row.ROC_AUC_plot + 0.01, f"{row.ROC_AUC_plot:.2f}", ha='center', fontsize=9, color='red')

plt.title("Ranking de Modelos: F1 (barras) e ROC AUC (linha vermelha)")
plt.ylabel("F1 / ROC AUC")
plt.xlabel("Modelo")
plt.ylim(0,1.05)
plt.show()


Salvar o melhor modelo

In [ ]:
joblib.dump(best_model, f"melhor_modelo_{ranking_final.iloc[0]['Modelo']}.joblib")
print("✅ Modelo salvo para uso futuro.")

Ranking completo com tempo de treino

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x="Modelo", y="F1", data=ranking_final, palette="pastel", edgecolor="black")

# Labels F1
for i, row in ranking_final.iterrows():
    plt.text(i, row.F1 + 0.01, f"{row.F1:.2f}", ha='center', fontsize=9)

# Adicionar linha de ROC AUC
for i, row in ranking_final.iterrows():
    if not pd.isna(row.ROC_AUC):
        plt.plot([i-0.2, i+0.2], [row.ROC_AUC]*2, color='red', lw=2)
        plt.text(i, row.ROC_AUC + 0.01, f"{row.ROC_AUC:.2f}", ha='center', fontsize=9, color='red')

# Adicionar tempo de treino como marcador
for i, row in ranking_final.iterrows():
    plt.text(i, 0.05, f"{row.Tempo:.1f}s", ha='center', fontsize=8, rotation=90, color='blue')

plt.title("Ranking de Modelos: F1 (barras), ROC AUC (linha vermelha) e Tempo de treino (azul)")
plt.ylabel("F1 / ROC AUC")
plt.xlabel("Modelo")
plt.ylim(0,1.05)
plt.show()


SHAP values para interpretação do melhor modelo

In [ ]:
import shap

# Criar explainer
explainer = shap.Explainer(best_model, X_train)
shap_values = explainer(X_test)

# Resumo global
shap.summary_plot(shap_values, X_test, plot_type="bar")  # importance plot
shap.summary_plot(shap_values, X_test)  # beeswarm plot

# Explicação individual (para o primeiro paciente)
shap.plots.waterfall(shap_values[0])


Fatores críticos de risco (visual)

In [ ]:
# Selecionar top 10 features SHAP
shap_importance = pd.Series(np.abs(shap_values.values).mean(0), index=X_test.columns)
top_features = shap_importance.sort_values(ascending=False).head(10)

plt.figure(figsize=(8,6))
top_features.sort_values().plot(kind='barh', color='orange')
plt.xlabel("Impacto médio absoluto (SHAP)")
plt.title("Top 10 fatores críticos de risco de doenças cardíacas")
plt.grid(axis='x')
plt.show()


Fatores críticos de risco (visual)

Gerando realtorio

In [ ]:
!pip install pdfkit jinja2
!apt-get install -y wkhtmltopdf  # necessário para PDF


Template de relatório HTML

In [ ]:
html_template = """
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <title>Relatório de Modelos - Risco de Doenças Cardíacas</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 30px;
            line-height: 1.5;
            color: #333333;
        }
        h1, h2, h3 {
            color: #2E4053;
        }
        h1 {
            text-align: center;
        }
        table {
            border-collapse: collapse;
            width: 100%;
            margin-bottom: 20px;
        }
        table, th, td {
            border: 1px solid #999;
        }
        th, td {
            padding: 8px;
            text-align: center;
        }
        img {
            display: block;
            margin: 10px auto;
            max-width: 90%;
            height: auto;
            border: 1px solid #ccc;
            padding: 5px;
            background-color: #f9f9f9;
        }
        p {
            font-size: 14px;
        }
    </style>
</head>
<body>
    <h1>Relatório de Modelos - Risco de Doenças Cardíacas</h1>

    <h2>1. Ranking de Modelos</h2>
    {{ ranking_table | safe }}

    <h2>2. Avaliação do Melhor Modelo: {{ best_model_name }}</h2>
    <p>
        Accuracy: {{ acc|round(3) }} |
        F1 Score: {{ f1|round(3) }} |
        ROC AUC: {{ roc_auc|round(3) }}
    </p>

    <h3>3. Matriz de Confusão</h3>
    <img src="{{ cm_img }}">

    <h3>4. Curva ROC</h3>
    <img src="{{ roc_img }}">

    <h3>5. Top Features</h3>
    <img src="{{ features_img }}">

    <h3>6. SHAP Summary Plot (Top 10 features)</h3>
    <img src="{{ shap_img }}">
</body>
</html>
"""




Salvar gráficos como imagens

In [ ]:
# ========================
# Matriz de Confusão
# ========================
cm_fig = "cm.png"
cm = confusion_matrix(y_test, best_model.predict(X_test))
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
plt.title("Matriz de Confusão")
plt.xlabel("Predito")
plt.ylabel("Real")
plt.savefig(cm_fig)
plt.close()

# ========================
# Curva ROC
# ========================
roc_fig = "roc.png"
y_score = None
if len(np.unique(y_test)) == 2:
    if hasattr(best_model, "predict_proba"):
        y_score = best_model.predict_proba(X_test)[:,1]
    elif hasattr(best_model, "decision_function"):
        y_score = best_model.decision_function(X_test)

if y_score is not None:
    RocCurveDisplay.from_predictions(y_test, y_score)
    plt.title("Curva ROC")
    plt.savefig(roc_fig)
    plt.close()

# ========================
# Top features (coef / importance)
# ========================
features_fig = "features.png"
plt.figure(figsize=(8,6))

# ========================
# Matriz de Confusão
# ========================
cm_fig = "cm.png"
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
plt.title("Matriz de Confusão")
plt.savefig(cm_fig)
plt.close()

# ========================
# Curva ROC
# ========================
roc_fig = "roc.png"
if len(np.unique(y_test)) == 2 and hasattr(best_model, "predict_proba"):
    RocCurveDisplay.from_estimator(best_model, X_test, y_test)
    plt.title("Curva ROC")
    plt.savefig(roc_fig)
    plt.close()

# ========================
# Top Features (coef / importance)
# ========================
features_fig = "features.png"
plt.figure(figsize=(8,6))

if hasattr(best_model, "coef_"):
    # Modelos lineares
    coefs = pd.Series(best_model.coef_[0], index=X_train.columns)
    top_features_model = coefs.abs().sort_values(ascending=False).head(10)
    colors = ['red' if coefs[f] > 0 else 'green' for f in top_features_model.index]
    top_features_model.sort_values().plot(kind='barh', color=colors)
elif hasattr(best_model, "feature_importances_"):
    # Modelos baseados em árvores
    feat_imp = pd.Series(best_model.feature_importances_, index=X_train.columns)
    top_features_model = feat_imp.sort_values(ascending=False).head(10)
    top_features_model.sort_values().plot(kind='barh', color='skyblue')

plt.title("Top 10 Features")
plt.xlabel("Importância")
plt.grid(axis='x')
plt.savefig(features_fig)
plt.close()

# ========================
# SHAP Top 10
# ========================
shap_fig = "shap.png"
explainer = shap.Explainer(best_model, X_train)
shap_values = explainer(X_test)

shap_importance = pd.Series(np.abs(shap_values.values).mean(0), index=X_test.columns)
top_shap = shap_importance.sort_values(ascending=False).head(10)

plt.figure(figsize=(8,6))
top_shap.sort_values().plot(kind='barh', color='orange')
plt.xlabel("Impacto médio absoluto (SHAP)")
plt.title("Top 10 fatores críticos (SHAP)")
plt.grid(axis='x')
plt.savefig(shap_fig)
plt.close()





Preencher template e gerar HTML/PDF

In [ ]:
!pip install weasyprint

In [ ]:
import os
from jinja2 import Template
from weasyprint import HTML

# =========================
# Caminhos absolutos das figuras
# =========================
cm_fig = os.path.abspath("cm.png")
roc_fig = os.path.abspath("roc.png")
features_fig = os.path.abspath("features.png")
shap_fig = os.path.abspath("shap.png")

# =========================
# Renderizar template HTML
# =========================
# html_template deve ser sua string de template Jinja2
template = Template(html_template)

html_out = template.render(
    ranking_table=ranking_final.to_html(index=False),  # tabela de ranking
    best_model_name=ranking_final.iloc[0]["Modelo"],  # nome do melhor modelo
    acc=accuracy_score(y_test, y_pred),
    f1=f1_score(y_test, y_pred, average='weighted'),
    roc_auc=roc_auc_score(y_test, y_proba) if hasattr(best_model, "predict_proba") else 0,
    cm_img=cm_fig,
    roc_img=roc_fig,
    features_img=features_fig,
    shap_img=shap_fig
)

# =========================
# Salvar HTML
# =========================
html_file = "relatorio_modelos.html"
with open(html_file, "w", encoding="utf-8") as f:
    f.write(html_out)

print(f"✅ Relatório HTML gerado: {html_file}")

# =========================
# Gerar PDF usando WeasyPrint
# =========================
pdf_file = "relatorio_modelos.pdf"
HTML(html_file).write_pdf(pdf_file)

print(f"✅ Relatório PDF gerado: {pdf_file}")




## 9. Engenharia de Atributos (Opcional)
# Exemplo: criar bins de idade
# df['age_bin'] = pd.cut(df['age'], bins=[29,40,50,60,70,80], labels=False)

Eliminação de características com baixa variância

In [ ]:
df.apply(lambda x: x.nunique(), axis=0)